In [ ]:
# Part 1: Email authentication for gmail
import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds


# Part 2: Email - Mechanism to get sent emails and extract body from from them.
import time
def extract_body(payload):
    """Extract the body text from the email payload."""
    if 'body' in payload and payload['body'].get('data'):
        return base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8')
    
    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain' and part['body'].get('data'):
                return base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
            
            # Recursive check for multipart emails
            if 'parts' in part:
                body = extract_body(part)
                if body:
                    return body
    
    return "(No body text found)"

def get_sent_messages(max_results=5000, max_count=None, search_query=None):
    """Get sent messages from Gmail with optional search filtering."""
    creds = authenticate_gmail()
    service = build('gmail', 'v1', credentials=creds)

    try:
        # Base query to get messages from sent folder
        query = "from:me"
        
        # Add additional search terms if provided
        if search_query:
            query += f" {search_query}"
        
        # Get message IDs that match the query with pagination
        message_ids = []
        next_page_token = None
        
        print("Fetching message IDs...")
        while True:
            results = service.users().messages().list(
                userId='me', 
                q=query, 
                maxResults=500,  # Maximum allowed per request
                pageToken=next_page_token
            ).execute()
            
            messages = results.get('messages', [])
            message_ids.extend(messages)
            
            # Print progress
            print(f"Found {len(message_ids)} messages so far...")
            
            # If we've reached our desired count, stop
            if max_count and len(message_ids) >= max_count:
                message_ids = message_ids[:max_count]
                break
                
            # Check if there are more pages
            next_page_token = results.get('nextPageToken')
            if not next_page_token:
                break
                
            # Sleep briefly to avoid rate limiting
            time.sleep(0.1)
        
        if not message_ids:
            print("No messages found.")
            return []
        
        print(f"Processing {len(message_ids)} messages...")
        
        # Process each message
        email_data = []
        for i, message in enumerate(message_ids):
            # Show progress every 50 messages
            if i % 50 == 0:
                print(f"Processing message {i+1} of {len(message_ids)}...")
                
            try:
                msg = service.users().messages().get(
                    userId='me', 
                    id=message['id'],
                    format='full'
                ).execute()
                
                # Extract headers
                headers = msg['payload']['headers']
                # subject = next((h['value'] for h in headers if h['name'].lower() == 'subject'), '(No Subject)')
                # to = next((h['value'] for h in headers if h['name'].lower() == 'to'), 'N/A')
                # date = next((h['value'] for h in headers if h['name'].lower() == 'date'), 'N/A')
                
                # Extract body (this can be complex due to MIME types)
                body = extract_body(msg['payload'])
                
                email_data.append({
                    # 'id': message['id'],
                    # 'subject': subject,
                    # 'to': to,
                    # 'date': date,
                    'body': body
                })
                
                # Sleep briefly to avoid rate limiting
                time.sleep(0.1)
                
            except HttpError as error:
                print(f'Error processing message {message["id"]}: {error}')
                continue
        
        return email_data
    
    except HttpError as error:
        print(f'An error occurred: {error}')
        return []


# Part 3: Searches through all sent messages - theres about 5000 of them. Took about 18 mins to run
# was last done on April 3 2025 at 5am PST.
# sent_messages = get_sent_messages()


# Part 4: Email parase - trim trailing \r\n from searches
final_public_uj_searches = []
for i_sent_msg in sent_messages:
    if not '\r\n' in i_sent_msg['body'][-5:]:
        pass
    elif '\r\n' in i_sent_msg['body']:
        final_public_uj_searches.append(i_sent_msg['body'].strip().upper())
        

# unique sorts of all searches organized neatly.
tots = {}
for i_sent_msg in final_public_uj_searches:
    tots[i_sent_msg] = tots.get(i_sent_msg,0)+1
public_uj_searches_unique_sorts = sorted(tots.items(), key=lambda x:x[1], reverse=True)
# public_uj_searches_unique_sorts

# Dump the file
import pickle
# with open('publicuj_searches_unique_sorts.pkl', 'wb') as f:
    # pickle.dump(public_uj_searches_unique_sorts, f)

In [ ]:
# Attempt to get notams
import requests

# Define your credentials
api_key = 'your_api_key_here'
username = 'your_username_here'
password = 'your_password_here'

# Set up headers and authentication
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}
auth = (username, password)

# API endpoint
url = 'https://api.faa.gov/s/communityapi/a083t00000XV7KUAA1/a083t00000xv7ku'

# Make the GET request
response = requests.get(url, headers=headers, auth=auth)

# Check the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f'Error {response.status_code}: {response.text}')
